# To-Dos
- [ ] Redo with full gemeente datasets
- [ ] Figure out how to create rows for totals by crime category
- [ ] Look up how to use transform vs. agg
- [ ] Keep overall crime per gemeente in final dataset; can then check violent crime as a percent of total crime in each gemeente

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math

# Data prep

## Import crime data

In [6]:
## Data extracted from: https://data.politie.nl/portal.html?_la=nl&_catalog=Politie&tableId=47013NED&_theme=114
## On: 12-07-2025

crime_data = 'RegisteredCrimes_AllCrimes_Gemeenten_2024.csv'
metadata = 'AlleMisdrijven_Metadata.csv'

meta_df = pd.read_csv(metadata, sep=';', index_col=1, header=1)
meta_df

,ID,ShortTitle,Identifier,Summary,Modified,ReasonDelivery,ExplanatoryText,Language,Catalog,Frequency,Period,ShortDescription,Description,DefaultPresentation,DefaultSelection,GraphTypes,OutputStatus,Source,MetaDataModified,SearchPriority
Title,,,,,,,,,,,,,,,,,,,,
"Geregistreerde misdrijven en aangiften; soort misdrijf, gemeente 2025",0,Misdrijven en aangiften per gemeente,47013NED,"Geregistreerde misdrijven, aangiften en intern...",2025-06-16T02:00:00,ActualiseringBijzonder,NaN,nl,Politie,Permaand,2012-2025,\nDeze tabel bevat cijfers over het aantal ger...,INHOUDSOPGAVE\n\n1. Toelichting\n2. Definities...,ts=1749630168797&graphtype=Table&r=RegioS&k=To...,$filter=((SoortMisdrijf eq '0.0.0 ')) and ((Pe...,"Table,Bar,Map",Regulier,&copy; Politie,2025-06-16T02:00:00,2.0
NaN,DataProperties,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,ID,ParentID,Type,Key,Title,Description,ReleasePolicy,Datatype,Unit,Decimals,Default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,NaN,Dimension,SoortMisdrijf,Soort misdrijf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,GeoDimension,RegioS,Regio's,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025 januari,2025MM01,NaN,Definitief,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025 februari,2025MM02,NaN,Definitief,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025 maart,2025MM03,NaN,Definitief,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
crime_df = pd.read_csv(crime_data, sep=';')
crime_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1
0,3492,0.0.0,GM1680,2012JJ00,925.0
1,3505,0.0.0,GM1680,2013JJ00,755.0
2,3518,0.0.0,GM1680,2014JJ00,664.0
3,3531,0.0.0,GM1680,2015JJ00,634.0
4,3544,0.0.0,GM1680,2016JJ00,616.0
...,...,...,...,...,...
263843,3847778,3.9.3,GM0999,2020JJ00,NaN
263844,3847791,3.9.3,GM0999,2021JJ00,NaN
263845,3847804,3.9.3,GM0999,2022JJ00,NaN
263846,3847817,3.9.3,GM0999,2023JJ00,NaN


## Extract crime metadata

In [9]:
print('Unique crime codes:')
print(crime_df['SoortMisdrijf'].unique())
print('Unqiue gemeente codes:')
print(crime_df['RegioS'].unique())
print('Unique year codes:')
print(crime_df['Perioden'].unique())

Unique crime codes:
['0.0.0 ' '1.1.1 ' '1.1.2 ' '1.2.1 ' '1.2.2 ' '1.2.3 ' '1.2.4 ' '1.2.5 '
 '1.3.1 ' '1.4.1 ' '1.4.2 ' '1.4.3 ' '1.4.4 ' '1.4.5 ' '1.4.6 ' '1.4.7 '
 '1.5.2 ' '1.6.1 ' '1.6.2 ' '1.6.3 ' '2.1.1 ' '2.2.1 ' '2.4.1 ' '2.4.2 '
 '2.5.1 ' '2.5.2 ' '2.6.1 ' '2.6.2 ' '2.6.3 ' '2.6.4 ' '2.6.5 ' '2.6.7 '
 '2.6.8 ' '2.6.9 ' '2.6.10' '2.6.11' '2.6.12' '2.6.13' '2.6.14' '2.7.2 '
 '2.7.3 ' '3.1.1 ' '3.1.2 ' '3.1.3 ' '3.2.1 ' '3.2.2 ' '3.3.2 ' '3.3.5 '
 '3.4.2 ' '3.5.2 ' '3.5.5 ' '3.6.4 ' '3.7.1 ' '3.7.2 ' '3.7.3 ' '3.7.4 '
 '3.9.1 ' '3.9.2 ' '3.9.3 ']
Unqiue gemeente codes:
['GM1680' 'GM0358' 'GM0197' 'GM0059' 'GM0482' 'GM0613' 'GM0361' 'GM0141'
 'GM0034' 'GM0484' 'GM1723' 'GM1959' 'GM0060' 'GM0307' 'GM0362' 'GM0363'
 'GM0200' 'GM0202' 'GM0106' 'GM0743' 'GM0744' 'GM0308' 'GM0489' 'GM0203'
 'GM0888' 'GM1954' 'GM0889' 'GM1945' 'GM1724' 'GM0893' 'GM0373' 'GM0748'
 'GM1859' 'GM1721' 'GM0753' 'GM0209' 'GM0375' 'GM0310' 'GM1728' 'GM0376'
 'GM0377' 'GM1901' 'GM0755' 'GM1681' 'GM0147' 'GM065

In [10]:
crime_codes = list(crime_df['SoortMisdrijf'].unique())
gemeente_codes = list(crime_df['RegioS'].unique())
year_codes = list(crime_df['Perioden'].unique())

In [11]:
crime_tuples = tuple(crime_codes)

crime_indices = []

for i in list(meta_df.index):
    if type(i) == float:
        continue
    elif i.startswith(crime_tuples):
        crime_indices.append(i)

crime_keys = []

for i in crime_indices:
    code_split = i.split(' ', maxsplit=1)
    crime_keys.append(code_split)


In [12]:
violent_crime = ['Zedendelicten','Moord, doodslag', 'Openlijk geweld (persoon)',
 'Bedreiging',
 'Mishandeling',
 'Straatroof',
 'Overval','Mensenhandel','Mensensmokkel', 'Kinderporno',
 'Kinderprostitutie']

In [13]:
crimekeys_df = pd.DataFrame(crime_keys,columns=['code','crime_name'])

violentcrime_df = crimekeys_df[crimekeys_df['crime_name'].isin(violent_crime)]

violentcrime_df.set_index(keys='code', inplace=True)

violentcrime_series = violentcrime_df['crime_name']

In [14]:
gmcodes_series = meta_df[meta_df['ID'].isin(gemeente_codes)]['ID']
yearcodes_series = meta_df[meta_df['ID'].isin(year_codes)]['ID']
gmcodes_series

Title
Aa en Hunze                    GM1680
Aalsmeer                       GM0358
Aalten                         GM0197
Achtkarspelen                  GM0059
Alblasserdam                   GM0482
                                ...  
Zwartewaterland                GM1896
Zwijndrecht                    GM0642
Zwolle                         GM0193
Buitenland                     GM0998
Gemeenten; niet in te delen    GM0999
Name: ID, Length: 344, dtype: object

In [15]:
violentcrime_dict = dict(violentcrime_series)
gm_dict = {}
for i, v in zip(gmcodes_series.index, gmcodes_series.values):
   gm_dict[v] = i

years_dict = {}
for i, v in zip(yearcodes_series.index, yearcodes_series.values):
    years_dict[v] = i


In [16]:
violentcrime_dict

{'1.4.1': 'Zedendelicten',
 '1.4.2': 'Moord, doodslag',
 '1.4.3': 'Openlijk geweld (persoon)',
 '1.4.4': 'Bedreiging',
 '1.4.5': 'Mishandeling',
 '1.4.6': 'Straatroof',
 '1.4.7': 'Overval',
 '1.6.3': 'Mensenhandel',
 '3.1.2': 'Mensensmokkel',
 '3.2.1': 'Kinderporno',
 '3.2.2': 'Kinderprostitutie'}

## Map crime, gemeente and year by codes to crime data

In [18]:
violentcrime_dict

{'1.4.1': 'Zedendelicten',
 '1.4.2': 'Moord, doodslag',
 '1.4.3': 'Openlijk geweld (persoon)',
 '1.4.4': 'Bedreiging',
 '1.4.5': 'Mishandeling',
 '1.4.6': 'Straatroof',
 '1.4.7': 'Overval',
 '1.6.3': 'Mensenhandel',
 '3.1.2': 'Mensensmokkel',
 '3.2.1': 'Kinderporno',
 '3.2.2': 'Kinderprostitutie'}

In [19]:
crime_df['SoortMisdrijf'] = crime_df['SoortMisdrijf'].apply(lambda x: x.strip())

In [20]:
crime_df['SoortMisdrijf'].unique()

array(['0.0.0', '1.1.1', '1.1.2', '1.2.1', '1.2.2', '1.2.3', '1.2.4',
       '1.2.5', '1.3.1', '1.4.1', '1.4.2', '1.4.3', '1.4.4', '1.4.5',
       '1.4.6', '1.4.7', '1.5.2', '1.6.1', '1.6.2', '1.6.3', '2.1.1',
       '2.2.1', '2.4.1', '2.4.2', '2.5.1', '2.5.2', '2.6.1', '2.6.2',
       '2.6.3', '2.6.4', '2.6.5', '2.6.7', '2.6.8', '2.6.9', '2.6.10',
       '2.6.11', '2.6.12', '2.6.13', '2.6.14', '2.7.2', '2.7.3', '3.1.1',
       '3.1.2', '3.1.3', '3.2.1', '3.2.2', '3.3.2', '3.3.5', '3.4.2',
       '3.5.2', '3.5.5', '3.6.4', '3.7.1', '3.7.2', '3.7.3', '3.7.4',
       '3.9.1', '3.9.2', '3.9.3'], dtype=object)

In [21]:
crime_df['crime_cat'] = crime_df['SoortMisdrijf'].map(violentcrime_dict)
crime_df['gemeente'] = crime_df['RegioS'].map(gm_dict)
crime_df['year'] = crime_df['Perioden'].map(years_dict)

In [22]:
crime_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1,crime_cat,gemeente,year
0,3492,0.0.0,GM1680,2012JJ00,925.0,NaN,Aa en Hunze,2012
1,3505,0.0.0,GM1680,2013JJ00,755.0,NaN,Aa en Hunze,2013
2,3518,0.0.0,GM1680,2014JJ00,664.0,NaN,Aa en Hunze,2014
3,3531,0.0.0,GM1680,2015JJ00,634.0,NaN,Aa en Hunze,2015
4,3544,0.0.0,GM1680,2016JJ00,616.0,NaN,Aa en Hunze,2016
...,...,...,...,...,...,...,...,...
263843,3847778,3.9.3,GM0999,2020JJ00,NaN,NaN,Gemeenten; niet in te delen,2020
263844,3847791,3.9.3,GM0999,2021JJ00,NaN,NaN,Gemeenten; niet in te delen,2021
263845,3847804,3.9.3,GM0999,2022JJ00,NaN,NaN,Gemeenten; niet in te delen,2022
263846,3847817,3.9.3,GM0999,2023JJ00,NaN,NaN,Gemeenten; niet in te delen,2023


In [23]:
crime_df['crime_cat'].unique()

array([nan, 'Zedendelicten', 'Moord, doodslag',
       'Openlijk geweld (persoon)', 'Bedreiging', 'Mishandeling',
       'Straatroof', 'Overval', 'Mensenhandel', 'Mensensmokkel',
       'Kinderporno', 'Kinderprostitutie'], dtype=object)

In [24]:
## Drop all rows were crime_cat is NaN after mapping
crime_df.dropna(subset=['crime_cat'], inplace=True)

In [25]:
crime_df

,ID,SoortMisdrijf,RegioS,Perioden,GeregistreerdeMisdrijven_1,crime_cat,gemeente,year
40248,590742,1.4.1,GM1680,2012JJ00,13.0,Zedendelicten,Aa en Hunze,2012
40249,590755,1.4.1,GM1680,2013JJ00,9.0,Zedendelicten,Aa en Hunze,2013
40250,590768,1.4.1,GM1680,2014JJ00,10.0,Zedendelicten,Aa en Hunze,2014
40251,590781,1.4.1,GM1680,2015JJ00,13.0,Zedendelicten,Aa en Hunze,2015
40252,590794,1.4.1,GM1680,2016JJ00,16.0,Zedendelicten,Aa en Hunze,2016
...,...,...,...,...,...,...,...,...
205707,2999528,3.2.2,GM0999,2020JJ00,NaN,Kinderprostitutie,Gemeenten; niet in te delen,2020
205708,2999541,3.2.2,GM0999,2021JJ00,NaN,Kinderprostitutie,Gemeenten; niet in te delen,2021
205709,2999554,3.2.2,GM0999,2022JJ00,NaN,Kinderprostitutie,Gemeenten; niet in te delen,2022
205710,2999567,3.2.2,GM0999,2023JJ00,NaN,Kinderprostitutie,Gemeenten; niet in te delen,2023


In [26]:
## Fill remaining NaN values in GeregistreerdeMisdrijven with 0
crime_df['GeregistreerdeMisdrijven_1'].fillna(value=0, inplace=True)

crime_df.rename(mapper={'RegioS':'Regions', 'Perioden':'Periods'}, axis=1, inplace=True)

C:\Users\chels\AppData\Local\Temp\ipykernel_15672\3992183514.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  crime_df['GeregistreerdeMisdrijven_1'].fillna(value=0, inplace=True)


In [27]:
crime_df.isna().sum()

ID                            0
SoortMisdrijf                 0
Regions                       0
Periods                       0
GeregistreerdeMisdrijven_1    0
crime_cat                     0
gemeente                      0
year                          0
dtype: int64

## Import & extract population data

In [29]:
## Extracted from: https://opendata.cbs.nl/statline/#/CBS/en/dataset/37259eng/table?ts=1752340527100
## https://opendata.cbs.nl/statline/portal.html?_la=en&_catalog=CBS&tableId=37259eng&_theme=1178
## On: 12-07-2025

# popdata = 'Bevolking_Gemeenten.csv'
## https://opendata.cbs.nl/statline/#/CBS/nl/dataset/37230ned/table?ts=1752318493150
popdata = 'Pop_NL_MajorCities.csv'


popdata_df = pd.read_csv(popdata, sep=';')

popdata_df.head()

,ID,Sex,Regions,Periods,PopulationOn31December_20
0,6461,T001038,GM0361,2012JJ00,94505
1,6462,T001038,GM0361,2013JJ00,94866
2,6463,T001038,GM0361,2014JJ00,95248
3,6464,T001038,GM0361,2015JJ00,107615
4,6465,T001038,GM0361,2016JJ00,108373


In [30]:
popdata_df.isna().sum()

ID                           0
Sex                          0
Regions                      0
Periods                      0
PopulationOn31December_20    0
dtype: int64

In [31]:
## Keep only 'bevolking aan het einde van de periode', RegioS and Perioden
popdata_df.drop(labels=['ID','Sex'], axis=1, inplace=True)

In [32]:
## Rename bevolking column to PopYearEnd
popdata_df.rename(mapper={'PopulationOn31December_20':'popyearend'}, axis=1, inplace=True)

In [33]:
popdata_df

,Regions,Periods,popyearend
0,GM0361,2012JJ00,94505
1,GM0361,2013JJ00,94866
2,GM0361,2014JJ00,95248
3,GM0361,2015JJ00,107615
4,GM0361,2016JJ00,108373
...,...,...,...
255,GM0855,2020JJ00,220505
256,GM0855,2021JJ00,224459
257,GM0855,2022JJ00,227707
258,GM0855,2023JJ00,229836


In [34]:
## Merge popdata_df to crime_df based on common RegioS and Perioden keys
crimepc_df = pd.merge(crime_df, popdata_df, on=['Regions', 'Periods'], how='left')

In [35]:
crimepc_df.rename({'GeregistreerdeMisdrijven_1':'registered_crimes'},axis=1, inplace=True)

In [141]:
crime_cities = crimepc_df[~crimepc_df['popyearend'].isna()]
pop_sort = crime_cities[crime_cities['year'] == '2024'].sort_values(by=['popyearend'], axis=0, ascending=False)

In [153]:
pop_sort.drop_duplicates(subset=['gemeente','year'], inplace=True)
pop_sort

,ID,SoortMisdrijf,Regions,Periods,registered_crimes,crime_cat,gemeente,year,popyearend
4679,658758,1.4.2,GM0363,2024JJ00,270.0,"Moord, doodslag",Amsterdam,2024,934526.0
12102,763506,1.4.3,GM0599,2024JJ00,343.0,Openlijk geweld (persoon),Rotterdam,2024,672960.0
14767,804570,1.4.4,GM0518,2024JJ00,1391.0,Bedreiging,'s-Gravenhage (gemeente),2024,568945.0
5563,670590,1.4.2,GM0772,2024JJ00,60.0,"Moord, doodslag",Eindhoven,2024,249035.0
30393,1029900,1.4.7,GM0855,2024JJ00,4.0,Overval,Tilburg,2024,230357.0
40364,2876040,3.2.1,GM0034,2024JJ00,5.0,Kinderporno,Almere,2024,229574.0
20422,885654,1.4.5,GM0268,2024JJ00,539.0,Mishandeling,Nijmegen,2024,189007.0
40871,2882826,3.2.1,GM0758,2024JJ00,7.0,Kinderporno,Breda,2024,188779.0
233,593856,1.4.1,GM0202,2024JJ00,153.0,Zedendelicten,Arnhem,2024,169364.0
28235,1001016,1.4.7,GM0392,2024JJ00,6.0,Overval,Haarlem,2024,168743.0


In [80]:
crimecities_sub = crime_cities[['gemeente','year','crime_cat','registered_crimes','popyearend']]
crimecities_sub

,gemeente,year,crime_cat,registered_crimes,popyearend
78,Alkmaar,2012,Zedendelicten,43.0,94505.0
79,Alkmaar,2013,Zedendelicten,44.0,94866.0
80,Alkmaar,2014,Zedendelicten,46.0,95248.0
81,Alkmaar,2015,Zedendelicten,52.0,107615.0
82,Alkmaar,2016,Zedendelicten,57.0,108373.0
...,...,...,...,...,...
48277,Tilburg,2020,Kinderprostitutie,1.0,220505.0
48278,Tilburg,2021,Kinderprostitutie,0.0,224459.0
48279,Tilburg,2022,Kinderprostitutie,0.0,227707.0
48280,Tilburg,2023,Kinderprostitutie,0.0,229836.0


In [38]:
crimecities_sub.isna().sum()

registered_crimes    0
crime_cat            0
gemeente             0
year                 0
popyearend           0
dtype: int64

## Total violent crime by city and year

In [73]:
# Calculate total crime per thousand for each gemeente
total_per_gemeente = crimecities_sub.groupby(['gemeente', 'year']).agg({'registered_crimes': 'sum'}).reset_index()
total_per_gemeente['crime_cat'] = 'Total'

,gemeente,year,registered_crimes,crime_cat,popyearend
0,'s-Gravenhage (gemeente),2012,6173.0,Total,505856.0
1,'s-Gravenhage (gemeente),2012,6173.0,Total,505856.0
2,'s-Gravenhage (gemeente),2012,6173.0,Total,505856.0
3,'s-Gravenhage (gemeente),2012,6173.0,Total,505856.0
4,'s-Gravenhage (gemeente),2012,6173.0,Total,505856.0
...,...,...,...,...,...
2855,Tilburg,2024,1446.0,Total,230357.0
2856,Tilburg,2024,1446.0,Total,230357.0
2857,Tilburg,2024,1446.0,Total,230357.0
2858,Tilburg,2024,1446.0,Total,230357.0


In [84]:
# Ensure unique combinations of gemeente and year by aggregating the population data
population_df_unique = crimecities_sub.drop_duplicates(subset=['gemeente', 'year'])

# Create a mapping Series from the unique population data
population_map = population_df_unique.set_index(['gemeente', 'year'])['popyearend']

population_map

# Map the population values to the crime DataFrame
total_per_gemeente['popyearend'] = total_per_gemeente.set_index(['gemeente', 'year']).index.map(population_map)


In [88]:
total_per_gemeente = total_per_gemeente[['gemeente','year','crime_cat','registered_crimes','popyearend']]
total_per_gemeente

,gemeente,year,crime_cat,registered_crimes,popyearend
0,'s-Gravenhage (gemeente),2012,Total,6173.0,505856.0
1,'s-Gravenhage (gemeente),2013,Total,6025.0,508940.0
2,'s-Gravenhage (gemeente),2014,Total,5633.0,514861.0
3,'s-Gravenhage (gemeente),2015,Total,5047.0,519988.0
4,'s-Gravenhage (gemeente),2016,Total,4770.0,524882.0
...,...,...,...,...,...
255,Tilburg,2020,Total,1620.0,220505.0
256,Tilburg,2021,Total,1524.0,224459.0
257,Tilburg,2022,Total,1603.0,227707.0
258,Tilburg,2023,Total,1450.0,229836.0


In [92]:
final_crimecities = pd.concat([crimecities_sub, total_per_gemeente], axis=0)

In [94]:
final_crimecities

,gemeente,year,crime_cat,registered_crimes,popyearend
78,Alkmaar,2012,Zedendelicten,43.0,94505.0
79,Alkmaar,2013,Zedendelicten,44.0,94866.0
80,Alkmaar,2014,Zedendelicten,46.0,95248.0
81,Alkmaar,2015,Zedendelicten,52.0,107615.0
82,Alkmaar,2016,Zedendelicten,57.0,108373.0
...,...,...,...,...,...
255,Tilburg,2020,Total,1620.0,220505.0
256,Tilburg,2021,Total,1524.0,224459.0
257,Tilburg,2022,Total,1603.0,227707.0
258,Tilburg,2023,Total,1450.0,229836.0


In [96]:
final_crimecities['crime_cat'].unique()

array(['Zedendelicten', 'Moord, doodslag', 'Openlijk geweld (persoon)',
       'Bedreiging', 'Mishandeling', 'Straatroof', 'Overval',
       'Mensenhandel', 'Mensensmokkel', 'Kinderporno',
       'Kinderprostitutie', 'Total'], dtype=object)

## Calculate per capita crime in each category & year, per gemeente

In [98]:
## Calculate new column crime_per_cap by dividing GeregistreerdeMisdrijven in each gemeente/category/year by per thousand of population per year/gemeente
final_crimecities['regcrimes_perthousand'] = final_crimecities['registered_crimes']/(final_crimecities['popyearend']/1000)

In [100]:
final_crimecities

,gemeente,year,crime_cat,registered_crimes,popyearend,regcrimes_perthousand
78,Alkmaar,2012,Zedendelicten,43.0,94505.0,0.455002
79,Alkmaar,2013,Zedendelicten,44.0,94866.0,0.463812
80,Alkmaar,2014,Zedendelicten,46.0,95248.0,0.482950
81,Alkmaar,2015,Zedendelicten,52.0,107615.0,0.483204
82,Alkmaar,2016,Zedendelicten,57.0,108373.0,0.525961
...,...,...,...,...,...,...
255,Tilburg,2020,Total,1620.0,220505.0,7.346772
256,Tilburg,2021,Total,1524.0,224459.0,6.789659
257,Tilburg,2022,Total,1603.0,227707.0,7.039748
258,Tilburg,2023,Total,1450.0,229836.0,6.308846


In [109]:
final_crimecities['year'] = final_crimecities['year'].astype('int64')
final_crimecities.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3120 entries, 78 to 259
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gemeente               3120 non-null   object 
 1   year                   3120 non-null   int64  
 2   crime_cat              3120 non-null   object 
 3   registered_crimes      3120 non-null   float64
 4   popyearend             3120 non-null   float64
 5   regcrimes_perthousand  3120 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 170.6+ KB


# Analysis

In [135]:
## Top cities, all violent crime 2024
# Create a boolean mask for rows where year is 2024 and crime_cat is 'Total'
bool_2024 = (final_crimecities['year'] == 2024) & (final_crimecities['crime_cat'] == 'Total')

# Use the boolean mask to filter the DataFrame
final_crimecities[bool_2024].sort_values(by=['regcrimes_perthousand'], axis=0, ascending=False)

,gemeente,year,crime_cat,registered_crimes,popyearend,regcrimes_perthousand
246,Rotterdam,2024,Total,6581.0,672960.0,9.779184
64,Amsterdam,2024,Total,8789.0,934526.0,9.404768
77,Arnhem,2024,Total,1362.0,169364.0,8.041851
12,'s-Gravenhage (gemeente),2024,Total,4411.0,568945.0,7.752946
155,Eindhoven,2024,Total,1753.0,249035.0,7.039171
207,Maastricht,2024,Total,824.0,125563.0,6.562443
259,Tilburg,2024,Total,1446.0,230357.0,6.277213
116,Breda,2024,Total,1183.0,188779.0,6.266587
25,'s-Hertogenbosch,2024,Total,1006.0,161530.0,6.227945
181,Leeuwarden,2024,Total,809.0,129973.0,6.224370


In [155]:
sexcrimes_2024 = (final_crimecities['year'] == 2024) & (final_crimecities['crime_cat'] == 'Zedendelicten')

final_crimecities[sexcrimes_2024].sort_values(by=['regcrimes_perthousand'], axis=0, ascending=False)

,gemeente,year,crime_cat,registered_crimes,popyearend,regcrimes_perthousand
233,Arnhem,2024,Zedendelicten,153.0,169364.0,0.903380
3158,Rotterdam,2024,Zedendelicten,600.0,672960.0,0.891583
2534,Nijmegen,2024,Zedendelicten,158.0,189007.0,0.835948
831,Deventer,2024,Zedendelicten,86.0,104301.0,0.824537
1091,Eindhoven,2024,Zedendelicten,189.0,249035.0,0.758929
207,Amsterdam,2024,Zedendelicten,685.0,934526.0,0.732992
623,Breda,2024,Zedendelicten,122.0,188779.0,0.646258
1351,'s-Gravenhage (gemeente),2024,Zedendelicten,357.0,568945.0,0.627477
246,Assen,2024,Zedendelicten,44.0,70392.0,0.625071
2066,Leeuwarden,2024,Zedendelicten,78.0,129973.0,0.600125


In [157]:
kinderporno_2024 = (final_crimecities['year'] == 2024) & (final_crimecities['crime_cat'] == 'Kinderporno')

final_crimecities[kinderporno_2024].sort_values(by=['regcrimes_perthousand'], axis=0, ascending=False)

,gemeente,year,crime_cat,registered_crimes,popyearend,regcrimes_perthousand
42314,Leeuwarden,2024,Kinderporno,22.0,129973.0,0.169266
41053,Delft,2024,Kinderporno,8.0,110173.0,0.072613
42535,Maastricht,2024,Kinderporno,8.0,125563.0,0.063713
40871,Breda,2024,Kinderporno,7.0,188779.0,0.037080
40338,Alkmaar,2024,Kinderporno,4.0,112896.0,0.035431
41339,Eindhoven,2024,Kinderporno,8.0,249035.0,0.032124
43406,Rotterdam,2024,Kinderporno,21.0,672960.0,0.031205
40481,Arnhem,2024,Kinderporno,5.0,169364.0,0.029522
40494,Assen,2024,Kinderporno,2.0,70392.0,0.028412
42782,Nijmegen,2024,Kinderporno,5.0,189007.0,0.026454


In [159]:
kinderprost_2024 = (final_crimecities['year'] == 2024) & (final_crimecities['crime_cat'] == 'Kinderprostitutie')

final_crimecities[kinderprost_2024].sort_values(by=['regcrimes_perthousand'], axis=0, ascending=False)

,gemeente,year,crime_cat,registered_crimes,popyearend,regcrimes_perthousand
44810,Alkmaar,2024,Kinderprostitutie,0.0,112896.0,0.0
44836,Almere,2024,Kinderprostitutie,0.0,229574.0,0.0
47878,Rotterdam,2024,Kinderprostitutie,0.0,672960.0,0.0
47254,Nijmegen,2024,Kinderprostitutie,0.0,189007.0,0.0
47072,Middelburg (Z.),2024,Kinderprostitutie,0.0,50152.0,0.0
47007,Maastricht,2024,Kinderprostitutie,0.0,125563.0,0.0
46838,Lelystad,2024,Kinderprostitutie,0.0,84713.0,0.0
46786,Leeuwarden,2024,Kinderprostitutie,0.0,129973.0,0.0
46383,'s-Hertogenbosch,2024,Kinderprostitutie,0.0,161530.0,0.0
46123,Haarlem,2024,Kinderprostitutie,0.0,168743.0,0.0


## Time series selected gemeenten of violent crime per capita

### Gemeentes with sharpest change in violent crime per capita (2012 -2024)

## Top gemeentes by violent crime per capita (2024)

### Homicides (moord, doodslag)

### Sex crime (zedendelicten, kinderporno, kinderprostitutie)